In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d yasserh/breast-cancer-dataset
!unzip \*.zip && rm *.zip

  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 54.0MB/s]
Archive:  breast-cancer-dataset.zip
  inflating: breast-cancer.csv       


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Step 1: Load Data
data = pd.read_csv('/content/breast-cancer.csv')

# Assuming 'diagnosis' is the target variable
X = data[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
          'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean']]
y = data['diagnosis']  # Target variable

# Step 2: Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Fit Gaussian Mixture Model
n_components = 5  # Number of components (clusters) in the Gaussian Mixture Model
gmm = GaussianMixture(n_components=n_components, random_state=42)
gmm.fit(X_train_scaled)

# Step 5: Estimate Distances using GMM
X_train_distances = -gmm.score_samples(X_train_scaled)
X_test_distances = -gmm.score_samples(X_test_scaled)

# Step 6: Model Training - GMD KNN
k = 5  # Number of neighbors
knn_model = KNeighborsClassifier(n_neighbors=k, weights='distance')
knn_model.fit(X_train_distances.reshape(-1, 1), y_train)

# Step 7: Model Evaluation
y_pred = knn_model.predict(X_test_distances.reshape(-1, 1))
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7280701754385965
Precision: 0.7398903508771929
Recall: 0.7280701754385965
F1 Score: 0.7312540610786226
Classification Report:
               precision    recall  f1-score   support

           B       0.81      0.73      0.77        71
           M       0.62      0.72      0.67        43

    accuracy                           0.73       114
   macro avg       0.72      0.73      0.72       114
weighted avg       0.74      0.73      0.73       114

